In [1]:
pip install kagglehub


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
madhavmalhotra_journal_entries_with_labelled_emotions_path = kagglehub.dataset_download('madhavmalhotra/journal-entries-with-labelled-emotions')

print('Data source import complete.')


Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

df = pd.read_csv(r'C:\Users\USER\Downloads\dataset\data.csv')

In [ ]:
df.replace({False: 0, True: 1}, inplace=True)
df.head()

In [ ]:
print((df.iloc[0]['Answer']))

In [ ]:
# 필요한 라이브러리 불러오기 (NLTK: 자연어 처리 도구 모음)
import nltk
from nltk.corpus import stopwords  # 불용어 사전 (의미 없는 단어들)
from nltk.tokenize import word_tokenize  # 문장을 단어로 나누는 함수
from nltk.sentiment import SentimentIntensityAnalyzer  # 감정 분석기 (VADER)
sia = SentimentIntensityAnalyzer()  # VADER 감정 분석기 객체 생성
import string  # 문자열 처리를 위한 표준 라이브러리

# CSV 파일 불러오기 (감정이 라벨링된 일기 데이터)
df = pd.read_csv('/kaggle/input/journal-entries-with-labelled-emotions/data.csv')

# True/False 값을 1/0으로 변환 (모델 학습을 위한 정수 처리)
df.replace({False: 0, True: 1}, inplace=True)

# 감정 점수를 저장할 리스트 초기화
list_pos = []       # 긍정 점수
list_neg = []       # 부정 점수
list_neu = []       # 중립 점수
list_compound = []  # 종합 감정 점수 (-1~1)

# 데이터프레임의 모든 행을 순회하면서 텍스트 정제 및 감정 점수 추출
for row in range(0, df.shape[0]):

    # 현재 행의 'Answer' 열(일기 문장)을 가져오기
    example_sent = df.iloc[row]['Answer']

    # 문장 부호 제거 (텍스트 정제)
    example_sent = example_sent.replace(r'.', '')
    example_sent = example_sent.replace(r',', '')
    example_sent = example_sent.replace(r'!', '')
    example_sent = example_sent.replace(r';', '')
    example_sent = example_sent.replace(r'?', '')
    example_sent = example_sent.replace(r'(', '')
    example_sent = example_sent.replace(r')', '')
    example_sent = example_sent.replace(r"'", '')

    # 영어 불용어 리스트 불러오기 (예: I, is, the 등)
    stop_words = set(stopwords.words('english'))

    # 일반적인 축약형 표현도 불용어로 추가
    list_contractions_common = {
        'arent', 'cant', 'couldnt', 'didnt', 'doesnt', 'dont',
        'hadnt', 'havent', 'shouldnt', 'wouldnt',
        'youve', 'youre', 'wont', 'werent', 'weve', 'wed',
        'theyre', 'Im'
    }
    stop_words.update(list_contractions_common)

    # 문장을 단어 단위로 분할 (토큰화)
    word_tokens = word_tokenize(example_sent)

    # 소문자로 바꾸고 stop_words에 포함된 단어 제거
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    # (의도상 오류) 위에서 생성한 filtered_sentence를 빈 리스트로 다시 초기화 → 앞의 작업이 무효됨
    filtered_sentence = []

    # 다시 토큰 순회하면서 불용어 제거 (소문자 변환 없음)
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)

    # 단어 리스트를 다시 하나의 문장으로 결합
    filtered_sentence = ' '.join(filtered_sentence)

    # 전처리된 문장으로 원래 'Answer' 열을 업데이트
    df.at[row, 'Answer'] = filtered_sentence

    # 감정 점수 추출: VADER가 반환하는 점수 중 각 항목만 따로 저장
    list_pos.append(sia.polarity_scores(filtered_sentence)['pos'])        # 긍정 점수
    list_neg.append(sia.polarity_scores(filtered_sentence)['neg'])        # 부정 점수
    list_neu.append(sia.polarity_scores(filtered_sentence)['neu'])        # 중립 점수
    list_compound.append(sia.polarity_scores(filtered_sentence)['compound'])  # 종합 점수

# 감정 점수 리스트를 원본 데이터프레임에 새로운 열로 추가
df['positive_score'] = list_pos
df['negative_score'] = list_neg
df['neutral_score'] = list_neu
df['compound_score'] = list_compound


In [ ]:
df.head(10)

In [ ]:
print((df.iloc[8]['Answer']))

In [ ]:
# VADER 감정 분석기를 nltk에서 불러옴
from nltk.sentiment import SentimentIntensityAnalyzer

# 감정 분석기 객체 생성
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(df.iloc[8]['Answer'])


In [ ]:
df.info()

In [ ]:
df.rename(columns={"Answer.f1.afraid.raw": "afraid",
                   "Answer.f1.angry.raw": "angry",
                   "Answer.f1.anxious.raw": "anxious",
                   "Answer.f1.ashamed.raw": "ashamed",
                   "Answer.f1.awkward.raw": "awkward",
                   "Answer.f1.bored.raw": "bored",
                   "Answer.f1.calm.raw": "calm",
                   "Answer.f1.confused.raw": "confused",
                   "Answer.f1.disgusted.raw": "disgusted",
                   "Answer.f1.excited.raw": "excited",
                   "Answer.f1.frustrated.raw": "frustrated",
                   "Answer.f1.happy.raw": "happy",
                   "Answer.f1.jealous.raw": "jealous",
                   "Answer.f1.nostalgic.raw": "nostalgic",
                   "Answer.f1.proud.raw": "proud",
                   "Answer.f1.sad.raw": "sad",
                   "Answer.f1.satisfied.raw": "satisfied",
                   "Answer.f1.surprised.raw": "surprised",
                   "Answer.t1.exercise.raw": "exercise",
                   "Answer.t1.family.raw": "family",
                   "Answer.t1.food.raw": "food",
                   "Answer.t1.friends.raw": "friends",
                   "Answer.t1.god.raw": "god",
                   "Answer.t1.health.raw": "health",
                   "Answer.t1.love.raw": "love",
                   "Answer.t1.recreation.raw": "recreation",
                   "Answer.t1.school.raw": "school",
                   "Answer.t1.sleep.raw": "sleep",
                   "Answer.t1.work.raw": "work",
                  }, inplace=True)

In [ ]:
df.head()

In [ ]:
print(df.iloc[0][12])
print(len(df))

In [ ]:
colname = df.columns[18]
print(colname)

In [ ]:
#18 emotions
#columns 1-18 (inclusive of both)
final_emotion_list = []
for row in range (0, len(df)):
    emotion_list = []
    for col in range (1, 19):
        if (df.iloc[row][col] == 1):
            emotion_list.append(df.columns[col])
    final_emotion_list.append(emotion_list)



In [ ]:
print(len(final_emotion_list))
print((final_emotion_list))

In [ ]:
df['Emotions Felt'] = final_emotion_list

In [ ]:
df.head()

In [ ]:
print(len(df))

In [ ]:
#11 topics
#columns 18-29 (inclusive of both)
#no topic lists greater than one for one person/entry, so we can just append it to one list
final_topic_list = []
for row in range (0, len(df)):
    #topic_list = []
    for col in range (19, 30):
        if (df.iloc[row][col] == 1):
            final_topic_list.append(df.columns[col])
#no topic lists greater than one, so

In [ ]:
print(len(final_topic_list))
print((final_topic_list))

In [ ]:
#There are two rows which don't have any reason selected, let's find them:

for row in range (0, len(df)):
    #topic_list = []
    count = 0
    for col in range (19, 30):
        if (df.iloc[row][col] == 1):
            count = count + 1
    if (count == 0):
        print(row)

In [ ]:
print(df.iloc[961][0])
# seems to be because of work
print(df.iloc[961][34])
print(df.iloc[961][33])

In [ ]:
print(df.iloc[980][0])
#seems to be because of god
print(df.iloc[980][34])
print(df.iloc[980][33])

In [ ]:
#now let's add the labels into the dataframe
#first one is for work which is col 29
df.iat[961,29] = 1
print(df.iloc[961][29])

In [ ]:
#now let's add the labels into the dataframe
#second one is for god which is col 23
print(df.iloc[980][23])
df.iat[980,23] = 1
print(df.iloc[980][23])

In [ ]:
#Now, let's rerun the code to get all the topics and the length should be correct
#11 topics
#columns 18-29 (inclusive of both)
#no topic lists greater than one for one person/entry, so we can just append it to one list
final_topic_list = []
for row in range (0, len(df)):
    #topic_list = []
    for col in range (19, 30):
        if (df.iloc[row][col] == 1):
            final_topic_list.append(df.columns[col])
#no topic lists greater than one, so

In [ ]:
print(len(final_topic_list))

In [ ]:
df['Emotion Topic'] = final_topic_list
df.head()

In [ ]:
df2 = df.copy()
df2.head()

In [ ]:
#dropping all the cols for emotions now that we have a comprehensive one at the end
df2 = df2.drop(df2.iloc[:, 1:19],axis = 1)
df2.head()

In [ ]:
#dropping all the cols for emotion topics now that we have a comprehensive one at the end
df2 = df2.drop(df2.iloc[:, 1:12],axis = 1)
df2.head()

In [ ]:
#swapping emotion topic and emotions felt columns
emotions_felt_list = df2['Emotions Felt']
#print(emotions_felt_list)
df2.drop(columns = ['Emotions Felt'], inplace = True)
#df2.head()
df2['Emotions Felt'] = emotions_felt_list
df2.head()

In [ ]:
print(df2.iloc[0][-1])

In [ ]:
df_copied = df2.copy()
df_copied.head()

In [ ]:
df_copied2 = df2.copy()
df_copied2.head()

In [ ]:
positive_emotions = ['satisfied', 'calm', 'happy', 'nostalgic', 'excited', 'proud', 'surprised']
negative_emotions = ['afraid', 'anxious', 'awkward', 'confused', 'ashamed', 'sad', 'angry', 'frustrated', 'disgusted', 'bored', 'jealous']

In [ ]:
print(set(['happy']).issubset(set(['happy', 'sad', 'anxious', 'excited'])))

In [ ]:
#864 entries in last column which have more than one entry in the emotions list
import random
positive_emotions = {'satisfied', 'calm', 'happy', 'nostalgic', 'excited', 'proud', 'surprised'}
negative_emotions = {'afraid', 'anxious', 'awkward', 'confused', 'ashamed', 'sad', 'angry', 'frustrated', 'disgusted', 'bored', 'jealous'}
p1 = {'satisfied', 'calm', 'happy', 'nostalgic'}
p2 = {'excited', 'proud', 'surprised'}

n1 = {'afraid', 'anxious', 'awkward', 'confused', 'ashamed', 'sad'}
n2 = {'angry', 'frustrated', 'disgusted', 'bored', 'jealous'}

for row in range (0, len(df_copied)):
    #topic_list = []
    if ((len(df_copied.iloc[row][-1])) > 1):
        pos = []
        neg = []
        for emotion in (df_copied.iloc[row][-1]):
            if (emotion in positive_emotions):
                pos.append(emotion)
            else:
                neg.append(emotion)
        if (len(pos) == len(neg)):
            c1 = 0
            c2 = 0
            c3 = 0
            c4 = 0
            for i in range (len(pos)):
                if (pos[i] in p1):
                    c1 = c1 + 1
                elif (pos[i] in p2):
                    c2 = c2 + 1
                elif (pos[i] in n1):
                    c3 = c3 + 1
                else:
                    c4 = c4 + 1
            max_val = max(c1, c2, c3, c4)
            if (max_val == c1):
                value = 'happy'
            elif (max_val == c2):
                value = 'excited'
            if (max_val == c3):
                value = 'anxious'
            else:
                value  = 'angry'
            df_copied.iat[row,-1] = [value]
        elif (len(pos) > len(neg)):
            c1 = 0
            c2 = 0
            for i in range (len(pos)):
                if (pos[i] in p1):
                    c1 = c1 + 1
                else:
                    c2 = c2 + 1
            if (c1 > c2):
                value = 'happy'
            else:
                value  = 'excited'
            df_copied.iat[row,-1] = [value]
        else:
            c1 = 0
            c2 = 0
            for i in range (len(neg)):
                if (neg[i] in n1):
                    c1 = c1 + 1
                else:
                    c2 = c2 + 1
            if (c1 > c2):
                value = 'anxious'
            else:
                value  = 'angry'
            df_copied.iat[row,-1] = [value]

    if (set(df_copied.iloc[row][-1]).issubset(p1)):
        df_copied.iat[row,-1] = 'happy'
    elif (set(df_copied.iloc[row][-1]).issubset(p2)):
        df_copied.iat[row,-1] = 'excited'
    elif (set(df_copied.iloc[row][-1]).issubset(n1)):
        df_copied.iat[row,-1] = 'anxious'
    else:
        df_copied.iat[row,-1] = 'angry'
df_copied.head(10)

In [ ]:
df_copied['Emotions Felt'].unique()

In [ ]:
#864 entries in last column which have more than one entry in the emotions list
import random
for row in range (0, len(df2)):
    #topic_list = []
    index = 0
    if ((len(df2.iloc[row][-1])) > 1):
        index = random.randrange(len(df2.iloc[row][-1]))
    df2.iat[row,-1] = (df2.iloc[row][-1])[index]
df2.head(10)

In [ ]:
df_copied.head()

In [ ]:
#importing libraries and modules
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
print(df2.iloc[:,6:7])

In [ ]:
print(df_copied.iloc[:,6:7])

In [ ]:
print((df_copied.iloc[:,6:7]).value_counts()['happy'])
print((df_copied.iloc[:,6:7]).value_counts()['angry'])
print((df_copied.iloc[:,6:7]).value_counts()['excited'])
print((df_copied.iloc[:,6:7]).value_counts()['anxious'])

In [ ]:
dfCOPY2 = df_copied.copy()

In [ ]:
df_copied['Emotions Felt'] = df_copied['Emotions Felt'].replace('happy',0)
df_copied['Emotions Felt'] = df_copied['Emotions Felt'].replace('excited',1)
df_copied['Emotions Felt'] = df_copied['Emotions Felt'].replace('angry',2)
df_copied['Emotions Felt'] = df_copied['Emotions Felt'].replace('anxious',3)

In [ ]:
df_copied.head(30)

In [ ]:
# example of oversampling a multi-class classification dataset
from pandas import read_csv
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
# define the dataset location
# split into input and output elements
X, y = df_copied.iloc[:,1:5], df_copied.iloc[:,6:7]
# label encode the target variable
y = LabelEncoder().fit_transform(y)
# transform the dataset
#oversample = SMOTE()
strategy = {0:3000, 1:3000, 2:3000, 3:3000}
oversample = SMOTE(sampling_strategy=strategy)
X, y = oversample.fit_resample(X, y)
# summarize distribution
counter = Counter(y)
for k,v in counter.items():
    per = v / len(y) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

In [ ]:
counter.items()

In [ ]:
y

In [ ]:
#ON DF_COPIED

from sklearn.ensemble import RandomForestClassifier #latest run took 1 hour 9min to run
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
rfc = RandomForestClassifier(n_estimators = 150)
rfc.fit(X_train, y_train)
y_pred_combo = rfc.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred_combo)
print(acc)


In [ ]:
#STEPS (don't run this piece of code)
#Need to take the test + classification of emotion topic
   # if there are multiple emotions topics, pick one at random?
# Extract scores out of it (emotion score)
#input it into the model with scores + topic for a final prediction

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
import string
from textblob import TextBlob


list_pos = []
list_neg = []
list_neu = []
list_compound = []

s1 = "Today was a very hard day for me because 3 days ago I lost my dad forever. And then 3 days later my mom is admitted into the hospital. My two biggest supporters are going away from me in just a few days"
s2 = "Two days ago, I was sitting in my living room. And it was raining. I personally don't like rain because when it rains everything gets wet and I hate looking at wet things. Also, a clean freak. So like when it rains my family members come from outside and they have dirt in their shoes and make the whole house dirty."
s3 = "Wow! I just landed my first internship today! Yay! I had a call with the recruiter and we finalized some details. I can't believe it!"
s4 = "Tomorrow is my mom's birthday and I planned a surprise party for her. I bought her favorite cake, snacks, and invited close friends and family! I can't wait to see her expression!"
s5 = "I am so stressed. I have three exams, 2 essays, one project, and a club proposal to submit all by the end of this week."
s6 = "Today was a very hard day for me because I was really feeling homesick, and I was missing my family and friends. I miss all our times together."
s7 = "It is so frustrating that I have not received the amazon package I ordered which had the gift for my friend's birthday. I have been waiting for weeks and it has still not arrived!"
s8 = "I forgot my friend's birthday, and I feel really bad now. How could I do this?"
s9 = "I lost my car keys, and I have been searching for hours and I still can't find them. I hate this!"

#also has a spell checker to account for certain spelling mistakes: ex. releeived and dissappointing
s10 = "I got so much work done today, and I feel really relieeved so now I don't have to worry about much for some time."
s11 = "I went for a walk with a friend to a boba shop, but it was closed, so that was dissappointing. Oh well...maybe next time."

example_sent = s3
example_sent = example_sent.replace(r'.', '')
example_sent = example_sent.replace(r',', '')
example_sent = example_sent.replace(r'!', '')
example_sent = example_sent.replace(r';', '')
example_sent = example_sent.replace(r'?', '')
example_sent = example_sent.replace(r'(', '')
example_sent = example_sent.replace(r')', '')
example_sent = example_sent.replace(r"'", '')

#print(example_sent)

#example_sent = example_sent.replace(r'[^\w\s]+,', '')
stop_words = set(stopwords.words('english'))
list_contractions_common = {'arent', 'cant', 'couldnt', 'didnt', 'doesnt', 'hadnt', 'havent', 'shouldnt', 'wouldnt', 'youve','youre','wont','werent', 'weve','wed', 'theyre', 'Im', 'its'}
stop_words.update(list_contractions_common)

word_tokens = word_tokenize(example_sent)

#print(word_tokens)
# converts the words in word_tokens to lower case and then checks whether
#they are present in stop_words or not
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#with no lower case conversion
filtered_sentence = []


for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

filtered_sentence = ' '.join(filtered_sentence)
#print(filtered_sentence)

final_sentence = filtered_sentence
#filtered_sentence = TextBlob(filtered_sentence)
#filtered_sentence = str(filtered_sentence.correct())

print(filtered_sentence)

pos_score = (sia.polarity_scores(filtered_sentence))['pos']
neg_score = (sia.polarity_scores(filtered_sentence))['neg']
neutral_score = (sia.polarity_scores(filtered_sentence))['neu']
compound_score = (sia.polarity_scores(filtered_sentence))['compound']


#emotion_topic = "work"#topic entered by user, may have to implement an algorithm to pick a topic if multiple are entered
#creating a list of the necessary important

#lists = [pos_score, neg_score, neutral_score, compound_score]
#a = lists.reshape(1, -1)
#then you feed lists into the model to predict

data = {'positive_score': [pos_score], 'negative_score': [neg_score], 'neutral_score': [neutral_score], 'compound_score': [compound_score]}

# Create DataFrame
df_unseen = pd.DataFrame(data)

predicted_score = rfc.predict(df_unseen)
#print(predicted_score)

#because the classification metrics were numbers, I need to convert them back to emotions
final_pred = ""
if (compound_score < 0):
    if (predicted_score == [2]):
        final_pred = 'angry'
    else:
        final_pred = 'anxious/sad'
elif (predicted_score == [0]):
    final_pred = 'happy'
elif (predicted_score == [1]):
    final_pred = 'excited'
elif (predicted_score == [2]):
    final_pred = 'angry'
else:
    final_pred = 'anxious/sad'

print(final_pred)


In [ ]:
X

#STEPS (don't run this piece of code)
#Need to take the test + classification of emotion topic
   # if there are multiple emotions topics, pick one at random?
# Extract scores out of it (emotion score)
#input it into the model with scores + topic for a final prediction

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
import string


list_pos = []
list_neg = []
list_neu = []
list_compound = []

    
example_sent = #sentence entered in the journal goes here
example_sent = example_sent.replace(r'.', '')
example_sent = example_sent.replace(r',', '')
example_sent = example_sent.replace(r'!', '')
example_sent = example_sent.replace(r';', '')
example_sent = example_sent.replace(r'?', '')
example_sent = example_sent.replace(r'(', '')
example_sent = example_sent.replace(r')', '')
example_sent = example_sent.replace(r"'", '')

#example_sent = example_sent[0].replace(r'[^\w\s]+,', '')
stop_words = set(stopwords.words('english'))
list_contractions_common = {'arent', 'cant', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'havent', 'shouldnt', 'wouldnt', 'youve','youre','wont','werent', 'weve','wed', 'theyre', 'Im'}
stop_words.update(list_contractions_common)

word_tokens = word_tokenize(example_sent)
# converts the words in word_tokens to lower case and then checks whether
#they are present in stop_words or not
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#with no lower case conversion
filtered_sentence = []


for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

filtered_sentence = ' '.join(filtered_sentence)


final_sentence = filtered_sentence

pos_score = (sia.polarity_scores(filtered_sentence))['pos']
neg_score = (sia.polarity_scores(filtered_sentence))['neg']
neutral_score = (sia.polarity_scores(filtered_sentence))['neu']
compound_score = (sia.polarity_scores(filtered_sentence))['compound']
    
emotion_topic = #topic entered by user, may have to implement an algorithm to pick a topic if multiple are entered
#creating a list of the necessary important
lists = [pos_score, neg_score, neutral_score, compound_score, emotion_topic]

#then you feed lists into the model to predict
predicted_score = rfc.predict(lists)

#because the classification metrics were numbers, I need to convert them back to emotions
final_pred = ""
if (predicted_score == 0):
    final_pred = 'happy'
elif (predicted_score == 1):
    final_pred = 'excited'
elif (predicted_score == 2):
    final_pred = 'angry'
else:
    final_pred = 'anxious'
    


In [ ]:
#ON DF_COPIED

from sklearn.ensemble import RandomForestClassifier #latest run took 1 hour 9min to run
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
rfc = RandomForestClassifier(n_estimators = 200)
rfc.fit(X_train, y_train)
y_pred_combo = rfc.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred_combo)
print(acc)


In [ ]:
print(rfc.predict[])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
knn_combination_f = KNeighborsClassifier(n_neighbors = 4)
knn_combination_f.fit(X_train, y_train)
y_pred_combo_f = knn_combination_f.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred_combo_f))

In [ ]:
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
nb2 = GaussianNB()
nb2.fit(X_train, y_train)
y_pred2 = nb2.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred2))

In [ ]:
df3 = df2.copy()
#happy = 1
#sad = 0
positive_labels = {"calm", "excited", "happy", "nostalgic", "proud", "satisfied", "surprised"}
#ok_labels = []
negative_labels = {"afriad", "angry", "anxious", "ashamed", "awkward", "bored","confused", "disgusted", "frustrated", "jealous", "sad"}
for row in range (0, len(df3)):
    #topic_list = []
    if ((df3.iloc[row][-1]) in positive_labels):
        df3.iat[row,-1] = "happy"
    else:
        df3.iat[row,-1] = "sad"
df3.head(10)

In [ ]:
#need to train one model to predict emotion topic
#need to train another to predict emotion felt

from sklearn.ensemble import RandomForestClassifier #latest run took 1 hour 9min to run
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:,1:5], df3.iloc[:,6:7], test_size=0.3, random_state=100)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
rfc = RandomForestClassifier(n_estimators = 143)
rfc.fit(X_train, y_train)
y_pred_combo = rfc.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred_combo)
print(acc)


In [ ]:
#ON DF_COPIED

from sklearn.ensemble import RandomForestClassifier #latest run took 1 hour 9min to run
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(df_copied.iloc[:,1:5], df_copied.iloc[:,6:7], test_size=0.3, random_state=100)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
rfc = RandomForestClassifier(n_estimators = 400)
rfc.fit(X_train, y_train)
y_pred_combo = rfc.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred_combo)
print(acc)


In [ ]:
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:,1:5], df3.iloc[:,6:7], test_size=0.3, random_state=100)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
nb2 = GaussianNB()
nb2.fit(X_train, y_train)
y_pred2 = nb2.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred2))

In [ ]:
#ON DF_COPIED

from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(df_copied.iloc[:,1:5], df_copied.iloc[:,6:7], test_size=0.3, random_state=100)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
nb2 = GaussianNB()
nb2.fit(X_train, y_train)
y_pred2 = nb2.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred2))

In [ ]:
max = 0
max_random_state = 0
for i in range (200):
    X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:,1:5], df3['Emotions Felt'], test_size=0.3, random_state=i)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    nb2 = GaussianNB()
    nb2.fit(X_train, y_train)
    y_pred2 = nb2.predict(X_test)
    if (metrics.accuracy_score(y_test, y_pred2) > max):
        max = metrics.accuracy_score(y_test, y_pred2)
        max_random_state = i
print(max)
print(max_random_state)

In [ ]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:,1:5], df3.iloc[:,6:7], test_size=0.3, random_state=30)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
logreg = LogisticRegression(solver = 'liblinear', random_state = 0)
logreg.fit(X_train, y_train)
pred = logreg.predict(X_test)
print(metrics.accuracy_score(y_test, pred))

In [ ]:
max = 0
max_random_state = 0

for i in range (200):
    X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:,1:5], df3['Emotions Felt'], test_size=0.3, random_state=i)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    logreg = LogisticRegression(solver = 'liblinear', random_state = 0)
    logreg.fit(X_train, y_train)
    pred = logreg.predict(X_test)

    if (metrics.accuracy_score(y_test, pred) > max):
        max = metrics.accuracy_score(y_test, pred)
        max_random_state = i

print(max)
print(max_random_state)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:,1:5], df3.iloc[:,6:7], test_size=0.3, random_state=20)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
knn_combination_f = KNeighborsClassifier(n_neighbors = 6)
knn_combination_f.fit(X_train, y_train)
y_pred_combo_f = knn_combination_f.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred_combo_f))

In [ ]:
#ON DF_COPIED
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(df_copied.iloc[:,1:5], df_copied.iloc[:,6:7], test_size=0.3, random_state=42)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
knn_combination_f = KNeighborsClassifier(n_neighbors = 10)
knn_combination_f.fit(X_train, y_train)
y_pred_combo_f = knn_combination_f.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred_combo_f))

In [ ]:
max = 0
max_random_state = 0

for i in range (200):
    X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:,1:5], df3['Emotions Felt'], test_size=0.3, random_state= i)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    knn_combination_f = KNeighborsClassifier(n_neighbors = 4)
    knn_combination_f.fit(X_train, y_train)
    y_pred_combo_f = knn_combination_f.predict(X_test)
    if (metrics.accuracy_score(y_test, y_pred_combo_f) > max):
        max = metrics.accuracy_score(y_test, y_pred_combo_f)
        max_random_state = i
print(max)
print(max_random_state)


In [ ]:
print(accuracy_array)

In [ ]:
# #ITERATIVE HYPERPARAMTER TUNING
# from sklearn.ensemble import RandomForestClassifier #latest run took 1 hour 9min to run
# accuracy_array = []
# max_accuracy = 0
# random_state_max = 0
# max_estimators = 0
# for i in range (1,100):
#     X_train, X_test, y_train, y_test = train_test_split(df3.iloc[:,1:5], df3['Emotions Felt'], test_size=0.3, random_state=i)
#     scaler = StandardScaler()
#     scaler.fit(X_train)
#     X_train = scaler.transform(X_train)
#     X_test = scaler.transform(X_test)
#     for x in range (20, 300):
#         rfc = RandomForestClassifier(n_estimators = x)
#         rfc.fit(X_train, y_train)
#         y_pred_combo = rfc.predict(X_test)
#         acc = metrics.accuracy_score(y_test, y_pred_combo)
#         if (acc > max_accuracy):
#             random_state_max = i
#             max_estimators = x
#             max_accuracy = acc
#         accuracy_array.append(acc)


# plt.figure(figsize=(10,6))
# plt.plot(range(1,17821),accuracy_array,color='blue', linestyle='dashed',marker='o',markerfacecolor='red', markersize=10)
# plt.title('Model Accuracy vs n-estimators vs random-state')
# plt.xlabel('n-estimators')
# plt.ylabel('Model Accuracy')
# print("Maximum accuracy",max(accuracy_array),"at N estimators =",max_estimators, "at random_state =", random_state_max)

In [ ]:
#need to train one model to predict emotion topic
#need to train another to predict emotion felt

from sklearn.ensemble import RandomForestClassifier #latest run took 1 hour 9min to run
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(df2.iloc[:,1:5], df2.iloc[:,6:7], test_size=0.3, random_state=10)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
rfc = RandomForestClassifier(n_estimators = 50)
rfc.fit(X_train, y_train)
y_pred_combo = rfc.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred_combo)
print(acc)


In [ ]:
#you have to take in an incoming stream of text, calculate all the score metrics, and then feed it into the model


In [ ]:
# cosine distance
# pick the most relevant emotion compared to the list of emotions

In [ ]:
data = pd.read_csv('/kaggle/input/journal-entries-with-labelled-emotions/data.csv')
data.head()

In [ ]:
data = data.iloc[:, 0:1]
data.head()

In [ ]:
data.info()

In [ ]:
# Load the regular expression library
import re
# Remove punctuation
data['Answer'] = \
data['Answer'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
data['Answer'] = \
data['Answer'].map(lambda x: x.lower())
# Print out the first rows of papers
data['Answer'].head()

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
long_string = ','.join(list(data['Answer'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]
data1 = data.Answer.values.tolist()
data_words = list(sent_to_words(data1))
# remove stop words
data_words = remove_stopwords(data_words)

#print(data_words[0])
print(data_words[:1][0][:30])

In [ ]:
#how often the word appears in the text

import gensim.corpora as corpora
# Create Dictionary
print(data_words[0])
id2word = corpora.Dictionary([data_words[0]])

print(id2word)

# Create Corpus


texts = data_words[0]
# Term Document Frequency
corpus = [id2word.doc2bow([text]) for text in texts]
# View
print(corpus)
#print(corpus[:1][0][:30])

In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
#compile all the keywords together in a set for the 10 topics, and just check if each topic (from dataset) is in it, and if not ....